In [81]:
import os
from dotenv import load_dotenv

# Load environment varibles
load_dotenv()

True

In [82]:
# Initialize llm model
from euriai.langchain import create_chat_model


llm_default = create_chat_model(
    api_key = os.getenv("EURI_API_KEY"),
    model = "gpt-4.1-nano",
    temperature = 0.2
)

In [83]:
response = llm_default.invoke("What is artificial intelligence?")
print(response.content)

Artificial intelligence (AI) refers to the simulation of human intelligence processes by computer systems and machines. These processes include learning (acquiring information and rules for using it), reasoning (using rules to reach conclusions), problem-solving, understanding language, perception (such as recognizing images or speech), and decision-making. AI can be categorized into narrow AI, which is designed for specific tasks (like voice assistants or recommendation systems), and general AI, which would have the ability to perform any intellectual task a human can do. The goal of AI research is to create systems that can operate autonomously and adapt to new situations, enhancing various aspects of technology and daily life.


In [84]:
import ast, math
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_react_agent,AgentExecutor
from langchain_core.runnables import RunnableLambda, RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_core.chat_history import InMemoryChatMessageHistory as ChatMessageHistory

#### Math Tool

In [85]:
# Define math tool
@tool("calculator", return_direct=True)
def calculator(expression: str) -> str:
    """Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions"""
    allowed_nodes =(
        ast.Expression,
        ast.UnaryOp,
        ast.unaryop,
        ast.BinOp,
        ast.operator,
        ast.Num,
        ast.Load,
        ast.pow,
        ast.FunctionDef,
        ast.Module,
        ast.Expr,
        ast.Call,
        ast.Name,
        ast.arguments,
        ast.args,
        ast.Constant
    )
    
    allowed_names = {k:v for k,v in vars(math).items() if not k.startswith("_")}
    allowed_names.update({"abs": abs, "round": round,"min": min,"max": max})
    node = ast.parse(expression, mode="eval")
    for n in ast.walk(node):
        if not isinstance(n, allowed_nodes):
            raise ValueError(f"Expression contains invalid node {type(n)}")
        if isinstance(n, ast.Name) and n.id not in allowed_names:
            raise ValueError(f"Expression contains invalid name {n.id}")
    code = compile(node, "<string>", "eval")
    return str(eval(code, {"__builtins__": {}}, allowed_names))

In [86]:
chat_prompt_math = ChatPromptTemplate.from_messages([(
    "system","you are a precise math assistant , you can use thse tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [87]:
tools_math = [calculator]
tools_math

[StructuredTool(name='calculator', description='Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions', args_schema=<class 'langchain_core.utils.pydantic.calculator'>, return_direct=True, func=<function calculator at 0x000001DEF5B64CC0>)]

In [88]:
agent_math = create_react_agent(
    llm=llm_default,
    prompt=chat_prompt_math,
    tools=tools_math
)

In [89]:
memory_math = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
)

In [65]:
math_exector = AgentExecutor(
    agent=agent_math,
    tools=tools_math,
    memory=memory_math,
    verbose=True,
    max_iterations=2,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the tools"
    ),
)

In [90]:
@tool("kb_search", return_direct=True)
def kb_search(query: str) -> str:
    """a mock function to search from a knowledge base"""
    knowledge_base = {
        "what is the capital of france": "The capital of France is Paris.",
        "who is the president of the united states": "The president of the United States is Joe Biden.",
        "what is the largest mammal": "The largest mammal is the blue whale.",
    }
    query = query.lower().strip("?")
    for key,answer in knowledge_base.items():
        if key in query:
            return answer
    return "I don't know the answer to that question."

In [91]:
tools_kb = [kb_search]
tools_kb

[StructuredTool(name='kb_search', description='a mock function to search from a knowledge base', args_schema=<class 'langchain_core.utils.pydantic.kb_search'>, return_direct=True, func=<function kb_search at 0x000001DEF5BE51C0>)]

In [ ]:
react_prompt_kb = ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant that can answer question based on your knowledge base and you can use the following tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final answer to the question that you are able to search from my knowledge base"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [93]:
agent_kb = create_react_agent(
    llm=llm_default,
    prompt=react_prompt_kb,
    tools=tools_kb
)

In [94]:
mem_kb = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
    k=5
)

In [95]:
kb_executor = AgentExecutor(
    agent=agent_kb,
    tools=tools_kb,
    memory=mem_kb,
    verbose=True,
    max_iterations=2,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the kb  tools"
    ),
)

In [96]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a router. Read the user message and output exactly one token:\n"
     "- MATH: if it asks for calculations.\n"
     "- KB: otherwise (tech education, RAG, agents, general know-how).\n"
     "Output only MATH or KB."),
    ("human", "{input}")
])

In [97]:
router_chain = router_prompt|llm_default|StrOutputParser()

In [74]:
def _dispatcher(inputs:dict):
    """it will take new inputs from human and it will also attach hitory from memory """
    user_msg = inputs["input"]
    choice = router_chain.invoke({"input": user_msg}).strip().upper()
    print(f"router choice : {choice}")
    if choice == "MATH" or choice == "Math" or choice == "math":
        return math_exector.invoke({"input": user_msg})
    elif choice == "KB" or choice == "Kb" or choice == "kb":
        return kb_executor.invoke({"input": user_msg})
    else:
        return "I can only answer math and kb related questions"

In [75]:
dispatcher = RunnableLambda(_dispatcher)
dispatcher

RunnableLambda(_dispatcher)

In [76]:
_sessions = {}

def _get_history(session_id:str):
    if session_id not in _sessions:
        _sessions[session_id] = ChatMessageHistory()
    return _sessions[session_id]

In [77]:
orchestrator = RunnableWithMessageHistory(
    runnable=dispatcher,
    get_session_history=_get_history,
    input_key="input",
    history_messages_key="history"
)

In [78]:
cfg = {"configurable":{"session_id":"user1"}}

In [79]:
print(orchestrator.invoke({"input":"what is the capital of france?"},config=cfg))

router choice : KB


> Entering new AgentExecutor chain...
Could not parse LLM output: `Thought: The question is asking for the capital city of France. I know from my general knowledge that the capital of France is Paris. However, I will confirm this information using the kb_search tool to ensure accuracy.

Action by using one of the tools access that you have [kb_search]: "capital of France"

Final answer: The capital of France is Paris.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the kb  toolsCould not parse LLM output: `Thought: The question asks for the capital of France. Based on my general knowledge, the answer is Paris. Since this is common knowledge, I do not need to use the kb_search tool for confirmation. 

Final answer: The capital of France is Paris.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSI

In [80]:
print(orchestrator.invoke({"input":"what is sin(90)"},config=cfg))

router choice : MATH


> Entering new AgentExecutor chain...
Could not parse LLM output: `Questions: what is sin(90)

thought: To find sin(90 degrees), I know that the sine of 90 degrees is 1.

Action by using one of the tools access that you have [calculator]: sin(90)

However, since the calculator function supports expressions with mathematical functions, I will evaluate sin(90 degrees). Note that if the calculator expects radians, I need to convert degrees to radians: 90 degrees = π/2 radians.

I'll evaluate sin(π/2).

Let's perform the calculation:

calculator("sin(3.141592653589793 / 2)")`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the toolsCould not parse LLM output: `Questions: what is sin(90)

thought: To accurately compute sin(90 degrees), I need to recognize that the calculator function expects input in radians. Since 90 degrees equals π/2 radians, 